In [0]:
#0858611 AI-model homework tensorflow_privacy

先安裝環境 使用pip install

In [1]:
pip install tensorflow_privacy

     |████████████████████████████████| 81kB 3.0MB/s 


In [0]:
import numpy as np
import tensorflow as tf
from keras import optimizers
#using keras tool
from privacy.analysis.rdp_accountant import compute_rdp
from privacy.analysis.rdp_accountant import get_privacy_spent
from privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

先測試沒有 differential_privacy 的方式訓練模型

In [9]:
def main():

  # Load training and test data.
  train_data, train_labels, test_data, test_labels = load_mnist()

  # Define a sequential Keras model
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                             strides=2,
                             padding='same',
                             activation='relu',
                             input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                             strides=2,
                             padding='valid',
                             activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

  GradientDescentOptimizer = tf.keras.optimizers.SGD
  optimizer = GradientDescentOptimizer(learning_rate=0.15)
  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)


  # Compile model with Keras
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  # Train model with Keras
  model.fit(train_data, train_labels,
            epochs=5,
            validation_data=(test_data, test_labels),
            batch_size=250)

  # Compute the privacy budget expended.

    
if __name__ == '__main__':
  main()

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.5596 - acc: 0.8206 - val_loss: 0.1153 - val_acc: 0.9626
Epoch 2/5
60000/60000 [==============================] - 2s 29us/sample - loss: 0.1151 - acc: 0.9640 - val_loss: 0.0739 - val_acc: 0.9773
Epoch 3/5
60000/60000 [==============================] - 2s 29us/sample - loss: 0.0732 - acc: 0.9771 - val_loss: 0.0516 - val_acc: 0.9839
Epoch 4/5
60000/60000 [==============================] - 2s 28us/sample - loss: 0.0582 - acc: 0.9820 - val_loss: 0.0458 - val_acc: 0.9859
Epoch 5/5
60000/60000 [==============================] - 2s 28us/sample - loss: 0.0503 - acc: 0.9844 - val_loss: 0.0413 - val_acc: 0.9874


準確率高達99%
且時間較短

使用differential_privacy來訓練模型

In [0]:
#following is adding differential_privacy

#first def epsilon parameter compute
noise_multiplier=1.1
def compute_epsilon(steps):
  """Computes epsilon value for given hyperparameters."""
  if noise_multiplier == 0.0:
    return float('inf')
  orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
  sampling_probability = 250 / 60000
  rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=1.1,
                    steps=steps,
                    orders=orders)
  return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]

In [0]:
def load_mnist():
  train, test = tf.keras.datasets.mnist.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  train_data = np.array(train_data, dtype=np.float32) / 255
  test_data = np.array(test_data, dtype=np.float32) / 255

  train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
  test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

  train_labels = np.array(train_labels, dtype=np.int32)
  test_labels = np.array(test_labels, dtype=np.int32)

  train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
  test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

  assert train_data.min() == 0.
  assert train_data.max() == 1.
  assert test_data.min() == 0.
  assert test_data.max() == 1.

  return train_data, train_labels, test_data, test_labels

In [8]:
##using optimizer to add differential_privacy

def main():

  # Load training and test data.
  train_data, train_labels, test_data, test_labels = load_mnist()

  # Define a sequential Keras model
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                             strides=2,
                             padding='same',
                             activation='relu',
                             input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                             strides=2,
                             padding='valid',
                             activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  
  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=1.0,
      noise_multiplier=1.1,
      num_microbatches=250,
      learning_rate=0.15)
    # Compute vector of per-example loss rather than its mean over a minibatch.
  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)

  # Compile model with Keras
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  # Train model with Keras
  model.fit(train_data, train_labels,
            epochs=5,
            validation_data=(test_data, test_labels),
            batch_size=250)

  # Compute the privacy budget expended.
  eps = compute_epsilon(5 * 60000 // 250)
  print('For delta=1e-5, the current epsilon is: %.2f' % eps)
    
if __name__ == '__main__':
  main()


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 130s 2ms/sample - loss: 1.5029 - acc: 0.5601 - val_loss: 0.7987 - val_acc: 0.7298
Epoch 2/5
60000/60000 [==============================] - 130s 2ms/sample - loss: 0.5950 - acc: 0.8163 - val_loss: 0.4805 - val_acc: 0.8595
Epoch 3/5
60000/60000 [==============================] - 130s 2ms/sample - loss: 0.4810 - acc: 0.8680 - val_loss: 0.4456 - val_acc: 0.8842
Epoch 4/5
60000/60000 [==============================] - 130s 2ms/sample - loss: 0.4680 - acc: 0.8858 - val_loss: 0.4371 - val_acc: 0.8988
Epoch 5/5
60000/60000 [==============================] - 130s 2ms/sample - loss: 0.4508 - acc: 0.8972 - val_loss: 0.4141 - val_acc: 0.9047
For delta=1e-5, the current epsilon is: 1.22


實際測試後未加上differential_privacy
準確率高達99% 且時間較短
而加上differential_privacy的
準確率僅90% 且時間增長許多